# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key

from api_keys import g_key


###### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../WeatherPy/WeatherPy.csv")
weather_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,Henties Bay,-22.12,14.28,69.01,24,0,1.01,NaN
1,Saint-François,46.42,3.91,59.00,87,0,2.24,FR
2,Ushuaia,-54.80,-68.30,26.60,79,0,4.70,AR
3,Mar del Plata,-38.00,-57.56,48.99,81,75,18.34,AR
4,Kruisfontein,-34.00,24.73,54.45,53,0,10.04,ZA


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

fig = gmaps.figure()

locations = weather_df[['Latitude','Longitude']]
weight = weather_df['Humidity']

heat_layer = gmaps.heatmap_layer(locations, weights= weight,dissipating=False,max_intensity=100,point_radius=5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
perfect_place = weather_df.loc[(weather_df['Max Temp'] < 80) &
                              (weather_df['Max Temp'] > 70) &
                              (weather_df['Humidity'] < 50) &
                              (weather_df['Cloudiness']<40)]
print(len(perfect_place))
perfect_place.dropna()
print(len(perfect_place))
perfect_place.head()

15
15


,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country
39,Yellowknife,62.46,-114.35,79.00,29,20,3.36,CA
56,Angren,41.02,70.14,75.47,27,8,4.90,UZ
136,Khatanga,71.97,102.50,75.06,41,29,2.57,RU
167,Sakakah,29.97,40.21,78.80,36,0,13.87,SA
181,Mount Isa,-20.73,139.50,78.80,31,14,12.75,AU


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = perfect_place

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

search_params = {"radius":5000,
                "types":"lodging",
                "keywords":"hotel",
                "key":g_key}



for index, row in hotel_df.iterrows():
    
    lat = row['Latitude']
    lng = row['Longitude']
    
    search_params['location'] = f"{lat},{lng}"
    
    resp = requests.get(base_url, params=search_params)
    
    resp_json = resp.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = resp_json["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Skipping location")

/Users/davispage/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/davispage/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Skipping location
Skipping location
Skipping location
Skipping location


In [6]:
#hotel_df = hotel_df.dropna()
hotel_df[hotel_df["Hotel Name"] != np.nan]

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
39,Yellowknife,62.46,-114.35,79.00,29,20,3.36,CA,The Explorer Hotel
56,Angren,41.02,70.14,75.47,27,8,4.90,UZ,Sardor Salimovich
136,Khatanga,71.97,102.50,75.06,41,29,2.57,RU,"Gostinitsa ""Zapolyar'ye"""
167,Sakakah,29.97,40.21,78.80,36,0,13.87,SA,Fakhamat Al Orjoana Apartment
181,Mount Isa,-20.73,139.50,78.80,31,14,12.75,AU,ibis Styles Mt Isa Verona
187,Yangi Marg`ilon,40.43,71.72,77.00,29,35,4.70,UZ,Ikathouse
242,Kodinsk,58.69,99.18,77.97,46,31,5.64,RU,NaN
264,Khandyga,62.67,135.60,79.57,33,0,1.90,RU,Apel'sin
406,Gilgit,35.92,74.31,76.73,32,21,0.47,PK,Tourist Hamlet Gilgit
447,Cedar City,37.68,-113.06,73.40,11,1,13.87,US,Hampton Inn Cedar City


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))